# Reference line for the trend/bootstrap

In [5]:
import random
import statistics
import pandas as pd
import numpy as np
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

In [ ]:
outpath=" "

In [ ]:
# load all the models
for year in range(1950,2021+1):
    globals()[f'model{year}'] = Word2Vec.load(f'/IRSF/Models/pd_{year}.model')

In [7]:
# define a function for bootstrap
def random_bias_year(old,young,length,simnum,path_output,filename):
    """Parameters:
    old: the word list for older adults
    young: the word list for young people
    length: the number of words in the list of domains (e.g., moral foundation, agiest attitude, etc.)"""
    yearlist = []
    meanlist = []
    lower95list = []
    upper95list = []
    
    models = {year: globals()[f'model{year}'] for year in range(1950, 2022)}
    
    for year in range(1950, 2021+1):
        model = models[year]
        vocabulary_year = model.wv.index_to_key
        ref_lists = [[vocabulary_year[i] for i in random.sample(range(len(vocabulary_year)), k=length)] for n in range(simnum+1)]
        similarities_A = [[model.wv.similarity(x, y) for x in ref_list for y in old if y in model.wv] for ref_list in ref_lists]
        similarities_B = [[model.wv.similarity(x, z) for x in ref_list for z in young if z in model.wv] for ref_list in ref_lists]
        biases = [statistics.mean(similarities_A[i]) - statistics.mean(similarities_B[i]) for i in range(simnum+1)]
        result = pd.DataFrame({'bias': biases})
        
        yearlist.append(year)
        meanlist.append(result['bias'].mean())
        lower95list.append(result['bias'].quantile(0.025))
        upper95list.append(result['bias'].quantile(0.975))
    
    result = pd.DataFrame({'year': yearlist, 'mean_similarity': meanlist,
                           '95percent_lower': lower95list, '95percent_upper': upper95list})
    result.to_csv(f"{path_output}{filename}.csv", index=False)
    return result

In [8]:
old=["老人", "老奶奶", "老爷爷", "曾祖母", "老祖母", "太奶奶", "老人家", "耄耋", "姥姥", "姥爷", "外公", "外婆", "奶奶", "爷爷", "花甲",
        "老婆婆", "古稀", "老母亲", "老父亲", "老伴儿", "老爷子", "老先生", "老太太", "老头", "大爷", "老大爷", "老伯伯", "老俩口"]
young=["小伙子", "青年人", "年轻人", "青年", "年青人", "年轻", "二十一二", "女孩子", "小青年", "少男少女", "女生", "姑娘", "男青年",
        "男孩子", "男生", "涉世未深", "十七八"]

In [1]:
care_vir=['保全', '友善', '友爱', '善,良', '助人为乐', '关心', '关怀', '爱护', '爱护', '关爱', '照顾', '庇护', '保护', '热心', '同情', '怜悯', '共情', '关照']
fare_vir=['正直', '公平', '公正', '公平正义', '合理', '公平合理', '客观', '正义', '权利', '一致', '平等', '互惠', '互利', '互惠互利', '平衡', '诚信']
loya_vir=['团体', '忠心', '忠诚', '忠心耿耿', '矢志不渝', '热爱祖国', '同胞', '爱国', '团结', '团结互助', '团结合作', '祖国', '社群', '集体', '成员', '服务', '奉献', '投身', '投身于'] 
auth_vir=['领导岗位', '权威', '服从', '尊崇', '合法', '责任', '尊敬', '敬重', '敬意', '崇敬', '敬仰', '敬佩', '秩序', '威严', '干部', '领导', '领导干部', '一把手']
sanc_vir=['崇高品质', '崇高', '纯净', '神圣', '清白', '纯洁', '圣洁', '光荣', '庄严', '风范', '光辉', '荣耀', '崇高精神', '高尚', '高尚情操', '赤诚', '情操']
care_vic=['戕害', '欺负', '伤害', '折磨', '践踏', '忍受', '遭受', '暴力', '暴虐', '残暴', '杀戮', '危害', '损害', '妨害', '毁坏', '毁灭', '滥用', '虐待', '无情']
fare_vic=['驱逐', '排外', '作假', '不公', '歧视', '不平', '偏颇', '盲从', '偏见', '偏执', '偏心', '偏爱', '排斥']
loya_vic=['敌伪', '汉奸', '背叛', '变节', '不忠', '敌对', '唾弃', '邪恶势力', '势力', '叛变', '特务', '叛徒', '叛乱']
auth_vic=['无序', '反叛', '违抗', '犯上', '颠覆', '反抗', '反复无常', '无所顾忌', '任,性', '不法', '非法', '不服', '煽动', '违法', '挑动', '混乱']
sanc_vic=['糟粕', '玷污', '传染', '罪恶', '肮脏', '恶心', '纳粹', '法西斯', '堕落', '泯灭', '松懈', '庸俗', '粗鄙', '粗俗', '低劣']

positive=['先进', '始终如一', '独立', '富有', '慷慨', '大方', '效率', '高效', '忙碌', '安全感', '强壮', '健康', '活跃', '好看', '合作', '配合', '乐观', '满足', '期待', '期盼', '灵活', '充满希望', '憧憬', '条理', '井井有条', '快乐', '友好', '整洁', '信任', '新来', '自立', '开明', '包容', '坚定', '宽容', '大度', '愉快的', '正常', '进攻性', '令人兴奋', '果断']
negative=['守旧', '反复无常', '依赖', '贫困', '自私', '低效率', '闲散', '不安', '惶恐', '虚弱', '病', '被动', '消极', '难看', '我行我素', '敷衍', '搪塞', '胡乱', '悲观', '不满', '贪婪', '放弃', '固执', '沮丧', '混乱', '悲伤', '咄咄逼人', '凶恶', '脏', '凌乱', '怀疑', '疑虑', '疑心', '依靠', '保守', '难以预料', '偏执', '讨厌', '古怪', '防备', '戒备', '无趣', '无聊', '犹豫', '优柔寡断', '犹疑', '优柔', '迟疑']
warm=['交际', '交友', '友善', '友爱', '友谊', '友好', '温暖', '慈爱', '慈祥', '和蔼', '和善', '可爱', '可亲', '活泼', '细腻', '热心', '欣然', '关怀', '礼貌', '关心', '温情', '愉悦', '幽默', '诙谐']
unwarm=['孤僻', '孤独', '寂寞', '冷漠', '冷淡', '冷酷', '强硬', '咄咄逼人', '讨厌', '烦人', '害羞', '麻木', '迟钝', '无情', '笨拙', '麻木不仁', '恶毒', '粗鲁', '野蛮', '无聊', '乏味', '沉闷']
competent=['能力', '胜任', '过硬', '做得好', '聪明', '聪明人', '精明', '机智', '聪慧', '智慧', '明智', '熟练', '娴熟', '思辨', '卓越', '杰出', '非凡', '洞察', '远见', '才智', '聪明才智']
incompetent=['无能', '花架子', '愚蠢', '目光短浅', '笨', '傻', '糊涂', '无知', '愚昧', '盲从', '笨拙', '低效', '不明智', '浑浑噩噩', '平庸', '庸碌', '庸官', '碌碌无为']

In [2]:
vir = care_vir+fare_vir+loya_vir+auth_vir+sanc_vir
vic = care_vic+fare_vic+loya_vic+auth_vic+sanc_vic

In [ ]:
boot_vir = random_bias_year(old,young,len(vir),1000,outpath,"null_vir")

In [ ]:
boot_vic = random_bias_year(old,young,len(vic),1000,outpath,"null_vic")

In [ ]:
boot_pos = random_bias_year(old,young,len(positive),1000,outpath,"null_pos")
boot_neg = random_bias_year(old,young,len(negative),1000,outpath,"null_neg")

In [ ]:
boot_warm = random_bias_year(old,young,len(warm),1000,outpath,"null_warm")
boot_unwarm = random_bias_year(old,young,len(unwarm),1000,outpath,"null_unwarm")
boot_competent = random_bias_year(old,young,len(competent),1000,outpath,"null_comp")
boot_incompetent = random_bias_year(old,young,len(incompetent),1000,outpath,"null_incomp")